In [1]:
!pip install wikipedia
!pip install spacy
!pip3 install scikit-learn
!pip3 install pandas
!pip3 install sentence-transformers


You should consider upgrading via the '/home/andre/miniconda3/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/andre/miniconda3/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/andre/miniconda3/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/andre/miniconda3/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/andre/miniconda3/bin/python3 -m pip install --upgrade pip' command.


In [2]:
doc_set = {}
doc_id = ""
doc_text = ""
with open('./CISI.ALL') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
doc_count = 0
for l in lines:
    if l.startswith(".I"):
        doc_id = int(l.split(" ")[1].strip())-1
    elif l.startswith(".X"):
        doc_set[doc_id] = doc_text.lstrip(" ")
        doc_id = ""
        doc_text = ""
    else:
        doc_text += l.strip()[3:] + " " # The first 3 characters of a line can be ignored.    

        
### Processing QUERIES
with open('./CISI.QRY') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
    
qry_set = {}
qry_id = ""
for l in lines:
    if l.startswith(".I"):
        qry_id = int(l.split(" ")[1].strip()) -1
    elif l.startswith(".W"):
        qry_set[qry_id] = l.strip()[3:]
        qry_id = ""

### Processing QRELS
rel_set = {}
with open('./CISI.REL') as f:
    for l in f.readlines():
        qry_id = int(l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0]) -1
        doc_id = int(l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1])-1
        if qry_id in rel_set:
            rel_set[qry_id].append(doc_id)
        else:
            rel_set[qry_id] = []
            rel_set[qry_id].append(doc_id)

print(rel_set.keys())

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 38, 40, 41, 42, 43, 44, 45, 48, 49, 51, 53, 54, 55, 56, 57, 60, 61, 64, 65, 66, 68, 70, 75, 78, 80, 81, 83, 89, 91, 94, 95, 96, 97, 98, 99, 100, 101, 103, 108, 110])


In [3]:
## Here we check some statistics and info of CISI dataset
import numpy as np
print('Read %s documents, %s queries and %s mappings from CISI dataset' % 
      (len(doc_set), len(qry_set), len(rel_set)))

number_of_rel_docs = [len(value) for key, value in rel_set.items()]
print('Average %.2f and %d min number of relevant documents by query ' % 
      (np.mean(number_of_rel_docs), np.min(number_of_rel_docs)))

print('Queries without relevant documents: ', 
      np.setdiff1d(list(qry_set.keys()),list(rel_set.keys())))

Read 1460 documents, 112 queries and 76 mappings from CISI dataset
Average 40.97 and 1 min number of relevant documents by query 
Queries without relevant documents:  [ 35  37  39  46  47  50  52  58  59  62  63  67  69  71  72  73  74  76
  77  79  82  84  85  86  87  88  90  92  93 102 104 105 106 107 109 111]


In [4]:
def mean_reciprocal_rank(bool_results, k=10):
    """Score is reciprocal of the rank of the first relevant item
    First element is 'rank 1'.  Relevance is binary (nonzero is relevant).
    Example from http://en.wikipedia.org/wiki/Mean_reciprocal_rank
    >>> rs = [[0, 0, 1], [0, 1, 0], [1, 0, 0]]
    >>> mean_reciprocal_rank(rs)
    0.61111111111111105

    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Mean reciprocal rank
    """
    bool_results = (np.atleast_1d(r[:k]).nonzero()[0] for r in bool_results)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in bool_results])


def mean_average_precision(bool_results,queries, K=10):
    """Precision and recall are single-value metrics based on the whole list of documents returned by the system. For systems that return a ranked sequence of documents, it is desirable to also consider the order in which the returned documents are presented. By computing a precision and recall at every position in the ranked sequence of documents, one can plot a precision-recall curve, plotting precision {\displaystyle p(r)}p(r) as a function of recall {\displaystyle r}r. Average precision computes the average 
    value of {\displaystyle p(r)}p(r) over the interval from {\displaystyle r=0}r=0 to {\displaystyle r=1}r=1:[3]
    Example from https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Mean_average_precision
    >>> rs = [[0, 0, 1], [0, 1, 0], [1, 0, 0]]
    >>> mean_reciprocal_rank(rs)
    0.61111111111111105

    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Mean reciprocal rank
    """
    result = 0
    for query in range(len(bool_results)):
        numerator = 0
        relevant_retrieved = np.count_nonzero(bool_results[query][:])
        if relevant_retrieved>0:
            for k in range(K):
                precision = np.count_nonzero(bool_results[query][:k+1])/len(bool_results[query][:k+1])
                # print(bool_results[query][:k+1],precision)
                numerator += bool_results[query][k] * precision
            result += numerator/relevant_retrieved
        
        
    return result / len(queries)


In [9]:
from components import PassageRanker


import time

binaries = []
query_idxs = []
index = 0
start_time = time.time()
for query, docs in rel_set.items():
    ranker = PassageRanker()
    results = ranker.rank(doc_set,qry_set[query],method='BERT')
    binary = []
    
    for i in range(10):
        if results[i][2] in docs:
            binary.append(1)
        else:
            binary.append(0)
    query_idxs.append(docs)
    binaries.append(binary)
    index+=1
    if index == 100:
        break
print("--- %s seconds ---" % (time.time() - start_time))
print(mean_reciprocal_rank(binaries))
print(mean_average_precision(binaries,query_idxs))

"""
TF-IDF
0.3828895154553049
0.3511450749890599

ST
0.6311925647451964
0.5885293073125938

vw2
0.30506996658312446
0.2577104218880535

BERT
0.07833333333333334
0.07833333333333334

BM25 b/k1

0.1/0.1
0.4325031328320802
0.387323865218602

0.25/0.25
0.41502715121136174
6940267335005

0.5/0.5
0.41985693400167085
0.36355158730158743

0.75/0.75
0.40587928153717623
0.33531830568484705

1.0/1.0
0.22340225563909774
0.20742298454469502

0.1/1.25
0.4114557226399332
0.37589279498349043

0.1/1.5
0.41779970760233914
0.3834560856506345

0.1/1.5
0.4153769841269841
0.3779492630385487

1/0.1
0.41558061821219716
0.371187306062776

1.25/0.1
0.4089129072681703
0.36275450531089626

1.75/0.1
0.3821063074352548
0.32777660918168433

2/0.1
0.335030284043442
0.30054389445836815

1.2/1.5
0.10955513784461153
0.10331035923141185

0.75/1.2
0.3719037176274018
0.3054108485499463

0.75/2
0.3353905597326651
0.27498120300751894

0/0
0.0888157894736842
0.09464285714285714



"""


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.se